In [ ]:
import os
import random
import numpy as np
import pandas as pd
from math import log
import multiprocessing as mp
from scipy.stats import mannwhitneyu

# 1. Import data:
## 1.1 one excel file with two sheets, containing data from four mice of one experiment:
#### ------ Matrix: 
       * gene expression data, processed by 1) cellranger aggr; 2) removal of low quality cells; 3) log2(TPM+1) normalization; 4) retaining genes with >= 3UMI in >= 5% cells
       * row index is cellular ID tagged with mouse ID; column index is gene ENSEMBL ID
#### ------ Bridge:
       * molecular bridge information obtained from processing of PacBio data
       * row index is the same as the Matrix sheet, column is "TBCID", i.e., tracking barcode

In [ ]:
with pd.ExcelFile('inputs/pooled_matrix_and_bridge.xlsx') as xls:
    matrix = pd.read_excel(xls, sheet_name='Matrix', index_col=0, header=0)
    bridge = pd.read_excel(xls, sheet_name='Bridge', index_col=0, header=0)
print(matrix.shape, bridge.shape)

## 1.2 cellular engraftment:
        * HSC activities measured by tracking barcode abundance in different cell populations
        * processed using methods described in Bramlett et al 2019 Nature Prot.

will be imported later on when needed

## 1.3 gene vocabulary:
        * stores gene name and gene ENSEMBL ID

In [ ]:
gene_vocabulary = pd.read_csv('inputs/gene_vocabulary.csv', index_col=None, header=0)
print('Number of genes:', len(gene_vocabulary.index))

# 2. Create scramble controls

In [ ]:
tbcids = list(set(bridge['TBCID'].values))
for i in range(100):
    frame = pd.DataFrame(columns=bridge.columns)
    for cell in matrix.index:
        k = random.sample(tbcids, 1)[0]
        frame.loc[cell, 'TBCID'] = k
    sheets['scramble'+str(i+1)] = frame
with pd.ExcelWriter(fil) as xlsx:
    matrix.to_excel(xlsx, sheet_name='Matrix')
    bridge.to_excel(xlsx, sheet_name='Bridge')
    for tab in sheets:
        sheets[tab].to_excel(xlsx, sheet_name=tab)

# 3. Calculate P values at every threshold for each mouse

In [5]:
def assignCellsToMice(cellset1, cellset2):
    result = {}
    for cell in cellset1:
        mouse = cell.split('-')[-1]
        if not mouse in result.keys():
            result[mouse] = [[], []]
        result[mouse][0].append(cell)
    for cell in cellset2:
        mouse = cell.split('-')[-1]
        if not mouse in result.keys():
            result[mouse] = [[], []]
        result[mouse][1].append(cell)
    return result

def calculate(four):
    scramble,ordered_values,ce_values,location = four
    print('Starting:', location, scramble, '...')
    if os.path.exists(os.path.join(location, scramble+'.csv')):
        return
    frame = pd.DataFrame()
    for i in range(len(ordered_values)):
        cells1 = ce_values[scramble][ce_values[scramble] <= ordered_values[i]].index
        cells2 = ce_values[scramble][ce_values[scramble] > ordered_values[i]].index
        cells_in_mice = assignCellsToMice(cells1, cells2)
        n = 0
        for mouse in cells_in_mice:
            locells,hicells = cells_in_mice[mouse]
            if len(locells) < 5 or len(hicells) < 5:
                continue
            n += 1
        if n < 4:
            continue
        for mouse in cells_in_mice:
            locells,hicells = cells_in_mice[mouse]
            for gid in matrix.columns:
                exp1 = matrix.loc[locells, gid].values
                exp2 = matrix.loc[hicells, gid].values
                pv1 = mannwhitneyu(exp1, exp2, alternative='less')[1]
                pv2 = mannwhitneyu(exp1, exp2, alternative='greater')[1]
                if np.isnan(pv1) or np.isnan(pv2):
                    continue
                fc = np.mean(exp1) - np.mean(exp2)
                frame.loc[gid, str(i+1)+'.'+mouse+'.fc'] = fc
                frame.loc[gid, str(i+1)+'.'+mouse+'.pv1'] = pv1
                frame.loc[gid, str(i+1)+'.'+mouse+'.pv2'] = pv2
    frame.to_csv(os.path.join(location, scramble+'.csv'), header=True, index=True)

def main(tab):
    subloc = 'individual-pvalues_' + tab
    if not os.path.exists(subloc):
        os.mkdir(subloc)
    print(tab)
    ce = {}
    with pd.ExcelFile('inputs/cellular_engraftments.xlsx') as xls:
        for mouse in xls.sheet_names:
            ce[mouse] = pd.read_excel(xls, sheet_name=mouse, index_col=0, header=0)[tab].dropna()
    ce_values = {}
    for scramble in scrambles:
        ce_values[scramble] = pd.Series(dtype='float64')
        for cell in scrambles[scramble].index:
            tbc = scrambles[scramble][cell]
            mx = ''
            mapped = False
            for mm in ce.keys():
                if tbc in ce[mm].index:
                    mapped = True
                    mx = mm
            if not mapped:
                continue
            ce_values[scramble][cell] = ce[mx][tbc]
    ordered_values = sorted(set(ce_values['Bridge'].values))
    
    print(' Begin multiprocessing...')
    pool = mp.Pool(25)
    pool.imap(calculate, [[sc, ordered_values, ce_values, subloc] for sc in scrambles])
    pool.close()
    pool.join()

In [ ]:
%%time
if __name__ == '__main__':
    for behavior in ('Gr', 'B', 'HSC', 'Plus', 'Bias'):
        main(tab=behavior)

# 4. Combine P values from four mice at each position

In [ ]:
%%time
for loc in os.listdir('.'):
    if not loc.startswith('individual-pvalues'):
        continue
    print(loc)
    if 'positioned-p.xlsx' in os.listdir(loc):
        continue
    frames = {}
    for fil in os.listdir(loc):
        if not fil.endswith('.csv'):
            continue
        scramble = fil.split('.')[0]
        print('', scramble)
        data = pd.read_csv(os.path.join(loc, fil), index_col=0, header=0)
        columns = {}
        for col in data.columns:
            a,b,c = col.split('.')
            if not a in columns.keys():
                columns[a] = [[],[],[]]
            columns[a][('fc','pv1','pv2').index(c)].append(col)
        frame = pd.DataFrame()
        for gid in data.index:
            numbers = data.loc[gid].dropna()
            for ii in sorted(columns.keys(), key=lambda x:int(x)):
                col1 = [co for co in columns[ii][0] if co in numbers.index]
                col2 = [co for co in columns[ii][1] if co in numbers.index]
                col3 = [co for co in columns[ii][2] if co in numbers.index]
                fcs = list(numbers[col1].values)
                pvs1 = list(numbers[col2].values)
                pvs2 = list(numbers[col3].values)
                if len(fcs) < 4:
                    continue
                frame.loc[gid, ii+'.cpv1'] = -log(combine_pvalues(pvs1)[1], 10)
                frame.loc[gid, ii+'.cpv2'] = -log(combine_pvalues(pvs2)[1], 10)
        frames[scramble] = frame
    with pd.ExcelWriter(os.path.join(loc, 'positioned-p.xlsx')) as xlsx:
        for scramble in frames:
            frames[scramble].to_excel(xlsx, sheet_name=scramble)

# 5. Use P values from exp and scrambles to calculate false positive score

In [ ]:
ALL_FPS = {}
for behavior in ('Gr', 'B', 'HSC', 'Plus', 'Bias'):
    xlsfile = os.path.join('individual-pvalues_%s'%behavior, 'positioned-p.xlsx')
    sheets = {}
    with pd.ExcelFile(xlsfile) as xls:
        for st in xls.sheet_names:
            sheets[st] = pd.read_excel(xls, sheet_name=st)
    fps = pd.DataFrame()
    maximum = {}
    for st in sheets:
        if st == 'Bridge':
            continue
        maximum[st] = {}
        for c in sheets[st].columns:
            maximum[st][c] = max(sheets[st][c].values)
    for gid in sheets['Bridge'].index:
        for col in sheets['Bridge'].columns:
            exp = sheets['Bridge'].loc[gid, col]
            scramble_values = [maximum[st][col] for st in maximum if col in maximum[st]]
            fps.loc[gid, col.split('.')[0] + '.fps' + col.split('pv')[1]] = 1.0 * len([v for v in scramble_values if v > exp]) / len(scramble_values)
    ALL_FPS[behavior] = fps

# 6. Count number of FPS that is below certain threshold, thus identify genes

In [ ]:
count_FPS = {}
for behavior in ALL_FPS:
    print(behavior, '...')
    for i,t in enumerate(('less', 'greater')):
        end = 'fps' + str(i+1)
        cols = [col for col in ALL_FPS[behavior].columns if col.endswith(end)]
        number = pd.DataFrame()
        column = ['#FPS < 0.05', '#FPS < 0.1', '#FPS < 0.2', '#all FPS']
        for gid in ALL_FPS[behavior].index:
            gname = gene_vocabulary['genename'][gene_vocabulary['geneid']==gid].values[0]
            scores = ALL_FPS[behavior].loc[gid, cols].values
            number.loc[gid, 'genename'] = gname
            number.loc[gid, column[0]] = len([v for v in scores if v <= .05])
            number.loc[gid, column[1]] = len([v for v in scores if v <= .1])
            number.loc[gid, column[2]] = len([v for v in scores if v <= .2])
            number.loc[gid, column[3]] = len(scores)
            number.loc[gid, 'best.fps'] = min(scores)
        number.sort_values(by=column, ascending=False, inplace=True)
        count_FPS[behavior + '.' + t] = number
with pd.ExcelWriter('Count_FPS.xlsx') as xlsx:
    for tab in count_FPS:
        count_FPS[tab].to_excel(xlsx, sheet_name=tab, index=True, header=True)